# Описание проекта

Банк хочет разработать кампанию лояльности по удержанию клиентов, но для этого ему необходимо выяснить основные причины оттока клиентов. Иными словами, нужно установить, чем ушедшие клиенты отличаются от лояльных и как между собой связаны различные признаки, определяющие клиентов.

### Столбцы таблицы:

- RowNumber — номер строки таблицы (это лишняя информация, поэтому можете сразу от неё избавиться)
- CustomerId — идентификатор клиента
- Surname — фамилия клиента
- CreditScore — кредитный рейтинг клиента (чем он выше, тем больше клиент брал кредитов и возвращал их)
- Geography — страна клиента (банк международный)
- Gender — пол клиента
- Age — возраст клиента
- Tenure — сколько лет клиент пользуется услугами банка
- Balance — баланс на счетах клиента в банке
- NumOfProducts — количество услуг банка, которые приобрёл клиент
- HasCrCard — есть ли у клиента кредитная карта (1 — да, 0 — нет)
- IsActiveMember — есть ли у клиента статус активного клиента банка (1 — да, 0 — нет)
- EstimatedSalary — предполагаемая заработная плата клиента
- Exited — статус лояльности (1 — ушедший клиент, 0 — лояльный клиент)


In [1]:
#Импорт нужных библиотек

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.express as px

In [2]:
#Посмотрим на данные

churn_data = pd.read_csv('data/churn.csv')
churn_data = churn_data.drop('RowNumber', axis=1)
churn_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/churn.csv'

## 1. Каково соотношение ушедших и лояльных клиентов? 
Покажем это на графике и дадим комментарии по соотношению.

In [ ]:
exited = churn_data['Exited'].value_counts() 
display(exited) # 0 - лояльные клиенты, 1 - ушедшие клиенты.

fig = plt.figure(figsize=(5,5))
axes = fig.add_axes([0,0,1,1])
axes.set_title('Соотношение лояльных и ушедших клиентов')
_, _, _ = axes.pie(
    exited,
    #labels=exited.index,
    autopct='%.1f%%',
    explode = [0, 0.1],
    labels=['Лояльные клиенты','Ушедшие клиенты']
)

Лояльных клиентов в четыре раза больше, чем ушедших.

## 2. Построим график, показывающий распределение баланса пользователей, у которых на счету больше 2 500 долларов. 
Опишем распределение и сделаем выводы.

In [ ]:
balance = churn_data['Balance'][churn_data['Balance'] > 2500] #Оставляем клиентов с балансом более 2500.

fig = plt.figure(figsize=(10,6))
axes = fig.add_axes([0,0,1,1])
axes.set_title('Распределение клиентов по размеру баланса на счёте')
axes.set_ylabel('Количество клиентов')
axes.set_xlabel('Баланс на счёте')
plt.hist(balance, bins=20)
plt.show()

Распределение очень близко к нормальному. Большинство клиентов держит на счетах от 90 до 150 тыс. долларов.

Наиболее многочисленна группа клиентов с балансом от 110 до 130 тыс.

Максимальный баланс на счёте - чуть более 250 тыс. 

## 3. Изучим распределение баланса клиента в разрезе признака оттока

Как различаются суммы на накопительном счёте ушедших и лояльных клиентов? Выдвигаем свои предположения, с чем это может быть связано, что может не устраивать ушедших клиентов в банке.

In [ ]:
# Сравним диаграммы boxplot

filtered_on_balance = churn_data[churn_data['Balance'] > 2500]
loyal = filtered_on_balance['Balance'][churn_data['Exited'] == 0]
departed = filtered_on_balance['Balance'][churn_data['Exited'] == 1]

fig = plt.figure(figsize=(7,8))
axes = fig.add_axes([0,0,1,1])
axes.set_title('Распределение баланса на счетах клиентов в зависимости от лояльности')
axes.set_ylabel('Баланс на счёте')
axes.set_xlabel('Лояльность')
plt.boxplot([loyal,departed], labels=['Лояльные клиенты','Ушедшие клиенты'])

plt.show()

На диаграммах выше заметно, что в группе ушедших клиентов значительно выше количество выбросов, а диапазон характерных сумм на счетах заметно более узкий, чем в группе лояльных клиентов. Т.е. группа ушедших клиентов менее однородна. 

Клиенов с небольшими суммами на счетах (менее 50000) и клиентов со значительными суммами (более 190000) могли не устраивать процентные ставки (зависящие от размера счёта).

В группе лояльных клиентов существенно ниже минимальный и заметно больше максимальный размер счёта.

Вполне возможно, клиенты ушли из-за различных нюансов обслуживания.

## 4. Изучим распределение возраста в разрезе признака оттока
В какой группе больше потенциальных выбросов? На какую возрастную категорию клиентов стоит обратить внимание банку?

In [ ]:
loyal = churn_data['Age'][churn_data['Exited'] == 0] # ушедшие клиенты
departed = churn_data['Age'][churn_data['Exited'] == 1] # лояльные клиенты

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14,5))

axes[0].hist(x=loyal, label=["Лояльные клиенты"], bins=20)
axes[0].set_title("Распределение по возрасту в группе лояльных клиентов")
axes[0].xaxis.set_tick_params(rotation=30)
axes[0].set_ylabel('Количество клиентов')
axes[0].set_xlabel('Возраст')

axes[1].hist(x=departed, label=["Ушедшие клиенты"], bins=20)
axes[1].set_title("Распределение по возрасту в группе ушедших клиентов")
axes[1].xaxis.set_tick_params(rotation=30)
axes[1].set_ylabel('Количество клиентов')
axes[1].set_xlabel('Возраст')

plt.show()

filtered_on_balance = churn_data
fig = px.box(filtered_on_balance, y="Age", x="Exited")
fig.show()

Очевидно, что, в целом, лояльные клиенты заметно моложе, чем ушедшие. Однако, в этой группе клиентов есть много выбросов. В эту группу входят самые пожилые и самые молодые клиенты банка.

Наиболее многочисленной группой среди лояльных клиентов являются люди от 31 до 41 года. Вероятно, банку стоит ориентироваться именно на этот возрастной диапазон в рекламе своих услуг. Или же расширять этот диапазон, включая в рекламу услуги для людей более молодого и более зрелого возрастов.

С другой стороны, в группе ушедших клиентов наиболее многочисленны люди в возрасте от 38 до 51 года. Банку имеет смысл предлагать группе клиентов такого возраста какие-либо дополнительные бонусы и услуги.

## 5. Построить график, который показывает взаимосвязь кредитного рейтинга клиента и его предполагаемой зарплаты
Добавим расцветку по признаку оттока клиентов. Какова взаимосвязь между признаками? Есть ли она вообще?

In [ ]:
loyal = churn_data[churn_data['Exited'] == 0] # лояльные клиенты
departed = churn_data[churn_data['Exited'] == 1] # ушедшие клиенты

fig = plt.figure(figsize=(14,10))
axes = fig.add_axes([0,0,1,1])
axes.scatter(
    x=loyal['EstimatedSalary'],
    y=loyal['CreditScore'],
    s=100,
    marker='o',
    c='green',
    label='Лояльные клиенты'
)
axes.scatter(
    x=departed['EstimatedSalary'],
    y=departed['CreditScore'],
    s=100,
    marker='o',
    c='red',
    label='Ушедшие клиенты'
)
axes.set_title('Взаимосвязь кредитного рейтинга и предполагаемой зарплаты у клиентов банка', fontsize=22)
axes.set_xlabel('Предполагаемая зарплата', fontsize=15)
axes.set_ylabel('Кредитный рейтинг', fontsize=15)
axes.set_ylim([330,900])
plt.legend(loc=2)
plt.show()

Явной взаимосвязи между кредитным рейтингом и предполагаемой зарплатой не наблюдается ни в первой, ни во второй группах клиентов.

## 6. Кто чаще уходит, мужчины или женщины? 
Построим график, который иллюстрирует это.

Процент ушедших клиентов в каждой группе рассчитаем как среднее по столбцу Exited (так как 1 — это ушедшие клиенты, а 0 — лояльные, среднее арифметическое по столбцу обозначает долю ушедших клиентов).

In [ ]:
departed = churn_data['Gender'][churn_data['Exited'] == 1] # ушедшие клиенты

fig = plt.figure(figsize=(5,5))
axes = fig.add_axes([0,0,1,1])
axes.set_title('Соотношение мужчин и женщин среди ушедших клиентов')
_, _, _ = axes.pie(
    departed.value_counts(), 
    autopct='%.1f%%',
    explode = [0, 0.1],
    labels=['Мужчины','Женщины']
)

Среди ушедших клиентов доля мужчин выше.

## 7. Как отток клиентов зависит от числа приобретённых у банка услуг? 

Для ответа на этот вопрос посроим многоуровневую столбчатую диаграмму.



In [ ]:
grouped = churn_data.groupby(by='NumOfProducts')['Exited'].sum()

fig = plt.figure(figsize=(12, 5))
axes = fig.add_axes([0, 0, 1, 1])
axes.bar(x = grouped.index, height = grouped);
axes.set_ylabel('Число ушедших клиентов', fontsize=16)
axes.set_xlabel('Число приобретённых у банка услуг', fontsize=16)
axes.set_xticks(range(0,5,1))
axes.set_title('Зависимость оттока клиентов от числа приобретённых услуг', fontsize=20)
plt.show()

Чем больше число приобретённых услуг, тем меньше количество ушедших клиентов. Банку следует забоиться о том, чтобы клиенты приобретали как можно больше услуг.

## 8. Как влияет наличие статуса активного клиента на отток клиентов? 

Построим диаграмму, иллюстрирующую это. Что можно было бы предложить банку, чтобы уменьшить отток клиентов среди неактивных?



In [ ]:
active = churn_data['Exited'][churn_data['IsActiveMember'] == 1] # активные клиенты
inactive = churn_data['Exited'][churn_data['IsActiveMember'] == 0] # неактивные клиенты

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14,5))

_, _, _ = axes[0].pie(
    active.value_counts(), 
    autopct='%.1f%%',
    explode = [0, 0.1],
    labels=['Лояльные клиенты','Ушедшие клиенты']
)
axes[0].set_title("Доли лояльных и ушедших среди активных клиентов")

_, _, _ = axes[1].pie(
    inactive.value_counts(), 
    autopct='%.1f%%',
    explode = [0, 0.1],
    labels=['Лояльные клиенты','Ушедшие клиенты']
)
axes[1].set_title("Доли лояльных и ушедших среди неактивных клиентов")

plt.show()

Среди неактивных клиентов доля ушедших ожидаемо выше (почти в два раза).

Рекомендация банку: больше работать с неактивными клиентами, стимулировать их пользоваться счётом.

## 9. В какой стране доля ушедших клиентов больше? 

Построим тепловую картограмму, которая покажет это соотношение на карте мира.



In [ ]:
grouped = churn_data.groupby(by='Geography')['Exited'].agg(['count','sum'])
grouped['Ratio'] = grouped['sum']/grouped['count']*100 # доля ушедших клиентов (число ушедших делим на общее число клиентов и умножаем на 100)
display(grouped)

fig = px.choropleth(
    data_frame=grouped, #датафрейм
    locations=grouped.index, #столбец с локациями
    locationmode = "country names", #режим сопоставления локаций с базой plotly
    color="Ratio", #от чего зависит цвет
    range_color=[16, 33], # интервал задан в соответствии с данными в колонке 'Ratio'
    title='Доля ушедших клиентов банка по странам', #заголовок
    width=900, #ширина 
    height=600, #высота
    color_continuous_scale='Reds' #палитра цветов
)

fig.show()

Доля ушедших клиентов больше всего в Германии.

Разный отток клиентов в разных странах может быть связан с:
- разным уровнем конкуренции в банковской сфере
- законами в данной конкретной стране
- разным уровнем жизни клиентов
- разным уровнем доходов и расходов
- отличиями в банковском обслуживании в разных странах
- количеством доступных банкоматов и офисов банка
- и т.д.

## 10. Переведём числовой признак CreditScore в категориальный

Для этого воспользуемся функцией get_credit_score_cat(), которая приведена ниже. Применим её к столбцу CreditScore и создадим новый признак CreditScoreCat — категории кредитного рейтинга.

Построим сводную таблицу, строками которой будут являться категории кредитного рейтинга (CreditScoreCat), а столбцами — количество лет, в течение которых клиент пользуется услугами банка (Tenure). В ячейках сводной таблицы должно находиться среднее по признаку оттока (Exited) — доля ушедших пользователей.

На основе полученной сводной таблицы построим тепловую карту с аннотацией. Поищем на тепловой карте категории клиентов, которые уходят чаще всего.

In [ ]:
#Функция для создания нового признака CreditScoreCat
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very_Poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"

In [ ]:
churn_data['CreditScoreCat'] = churn_data['CreditScore'].apply(get_credit_score_cat)

pivot = churn_data.pivot_table(
    values='Exited', # по умолчанию будет выводиться среднее значение (mean)
    index = 'CreditScoreCat',
    columns= 'Tenure'
)

fig = plt.figure(figsize=(14,8))
axes = fig.add_axes([0,0,1,1])
axes.set_title('Зависимость оттока клиентов от кредитного рейтинга и времени пользования услугами банка', fontsize=16)
axes.set_ylabel('Кредитный рейтинг', fontsize=14)
axes.set_xlabel('Время пользования услугами банка (лет)', fontsize=14)
axes.set_xticks(pivot.columns)
axes.set_yticks(range(5))
axes.set_yticklabels(labels=pivot.index)
plt.imshow(pivot, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

Чаще всего уходят клиенты с низким кредитным рейтингом, пользующиеся услугами банка 0 или 10 лет.